In [ ]:
# Credit to this kernel to this  competition - https://www.kaggle.com/manojprabhaakr/similar-duplicate-images-in-aptos-data. 

Above  kernel   has  selected  143  different  Hash_code(picture)  which  is  duplicated,  and  when  I  want  to drop  all   of  them,  I found  that   some  picture  has  the  same  diagnosis  and  same  Hash_code  simultaneously , which  means  their  diagnosis  are  not  conficted,  so  I  think  there   is  no   necessary  to  drop  them.
At   last ,what  I   select  is   two  or  more   picture    have   the   same  Hash_code(picture)  but   the  different  diagosis.

Above  is  important  ,please  read  it  first.

In [ ]:
import pandas as pd 
import os
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from PIL import Image
import imagehash

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
print(os.listdir("../input"))

Reading the CSV File

In [ ]:
train = pd.read_csv("../input/train.csv")

Checking the Unique Image Names. There are 3662 unique training examples

In [ ]:
train.id_code.nunique()
train.head()

Getting the path of the Image

In [ ]:
base_image_dir = os.path.join('..', 'input/') # Joining the base directory Input
train_dir = os.path.join(base_image_dir,'train_images/') # Training Directory Location
df = pd.read_csv(os.path.join(base_image_dir, 'train.csv')) # Reading the training file

df['path'] = df['id_code'].map(lambda x: os.path.join(train_dir,'{}.png'.format(x))) # Getting the path of the image

df.head(10) # Getting the top 10 records

Calculating the Hash, Shape, Mode, Length and Ratio of each image

In [ ]:
def getImageMetaData(file_path):
    with Image.open(file_path) as img:
        img_hash = imagehash.phash(img)
        return img.size, img.mode, img_hash

def get_train_input():
    train_input = df.copy()
        
    m = train_input.path.apply(lambda x: getImageMetaData(x))
    train_input["Hash"] = [str(i[2]) for i in m]
    train_input["Shape"] = [i[0] for i in m]
    train_input["Mode"] = [str(i[1]) for i in m]
    train_input["Length"] = train_input["Shape"].apply(lambda x: x[0]*x[1])
    train_input["Ratio"] = train_input["Shape"].apply(lambda x: x[0]/x[1])
    
    
    img_counts = train_input.path.value_counts().to_dict()
    train_input["Id_Count"] = train_input.path.apply(lambda x: img_counts[x])
    return train_input

train_input = get_train_input()


In [ ]:
train_input.head()


*If  two  or  more  row  has  same  diagnosis  and  same  Hash ,  keep  the  first*

In [ ]:
train_input=train_input.drop_duplicates(subset=['diagnosis','Hash'],keep='first')

train_input

In [ ]:
train_input1 = train_input[['Hash']] # Getting the Hash from the new data
train_input1['New']=1 # Creating a dummy column 1
train_input1.head()

In [ ]:
train_input2 = train_input1.groupby('Hash').count().reset_index() # Grouping the column by Hash to aggregate at Hash level
train_input2.tail()
train_input2.shape

In [ ]:
train_input2 = train_input2[train_input2['New']>1] # Filtering those instances where the hash is occuring multiple times

In [ ]:
train_input2.shape # Checking the shape

In [ ]:
train_input2 = train_input2.sort_values('Hash') # Sorting the data by Hash 
train_input2.tail(5) # Checking the top 5 records

In [ ]:
train_input.head()

In [ ]:
train_input.shape

In [ ]:
train_input2.head()

In [ ]:
train_input2.shape

In [ ]:
criterion = lambda row: row['Hash']  in  train_input2['Hash'].values.tolist()
Hash_in = train_input[train_input.apply(criterion, axis='columns')]
Hash_in.sort_values('Hash')
Hash_in.to_csv('Hash_in.csv', index=None)

If  two or  more  rows   in  train_input(the  orignal csv)  have the  same  Hash_code  and  different  diagnosis, they  will  all  appears  above  in  'Hash_in.csv'

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from IPython import display
import time

%matplotlib inline

PATH = "../input/train_images/a75bab2463d4.png"
image = mpimg.imread(PATH) # images are color images
plt.imshow(image);

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from IPython import display
import time

%matplotlib inline

PATH = "../input/train_images/1632c4311fc9.png"
image = mpimg.imread(PATH) # images are color images
plt.imshow(image);